# Data Cleaning with Pandas

In [1]:
import pandas as pd

## Catch-Up from Yesterday

## Reshaping a DataFrame

### `.pivot()` and `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

In [2]:
import pandas as pd
uci = pd.read_csv('heart.csv')

In [3]:
uci.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
uci.pivot(values = 'sex', columns = 'target').head()

target,0,1
0,NaN,1.0
1,NaN,1.0
2,NaN,0.0
3,NaN,1.0
4,NaN,0.0


In [5]:
uci.pivot_table(values = 'chol', index = 'sex', columns = 'target', aggfunc = 'mean')

target,0,1
sex,,
0,274.958333,256.750000
1,246.061404,230.989247


### Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`, `.melt()`

### `.join()`

In [6]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns = ['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns = ['age', 'HP'])

In [7]:
toy1.join(toy2.set_index('age'),
          on = 'age',
          lsuffix = '_1',
          rsuffix = '_2').head()

,age,HP_1,HP_2
0,63,142,100
1,33,47,200


### `.merge()`

In [8]:
ds_chars = pd.read_csv('ds_chars.csv', index_col = 0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [9]:
states = pd.read_csv('states.csv', index_col = 0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


In [10]:
ds_chars.merge(states,
               left_on='home_state',
               right_on = 'state',
               how = 'inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


### `pd.concat()`

Exercise: Look up the documentation on pd.concat (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) and use it to concatenate ds_chars and states.
<br/>
Your result should still have only five rows!

In [14]:
pd.concat([ds_chars, states], axis=1)

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,TX,alamo,Austin
2,alan,170,TX,DC,district,Washington
3,alison,300,DC,OH,buckeye,Columbus
4,rachel,200,TX,OR,beaver,Salem


### `pd.melt()`

Melting removes the structure from your DataFrame and puts the data in a 'variable' and 'value' format.

In [15]:
ds_chars.head()

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [16]:
pd.melt(ds_chars,
        id_vars=['name'],
        value_vars=['HP', 'home_state'])

,name,variable,value
0,greg,HP,200
1,miles,HP,200
2,alan,HP,170
3,alison,HP,300
4,rachel,HP,200
5,greg,home_state,WA
6,miles,home_state,WA
7,alan,home_state,TX
8,alison,home_state,DC
9,rachel,home_state,TX


## Bringing it all together with the Animal Shelter Data

Join the data from the [Austin Animal Shelter Intake dataset](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm) to the outcomes dataset by Animal ID.

Use the dates from each dataset to see how long animals spend in the shelter. Does it differ by time of year? By outcome?

The Url for the Intake Dataset is here: https://data.austintexas.gov/api/views/wter-evkm/rows.csv?accessType=DOWNLOAD

_Hints_ :
- import and clean the intake dataset first
- use apply/applymap/lambda to change the variables to their proper format in the intake data
- rename the columns in the intake dataset *before* joining
- create a new days-in-shelter variable
- Notice that some values in "days_in_shelter" column are NaN or values < 0 (remove these rows using the "<" operator and ~is.na())
- Use group_by to get some interesting information about the dataset

Make sure to export and save your cleaned dataset. We will use it in a later lecture!

use the notation `df.to_csv()` to write the `df` to a csv. Read more about the `to_csv()` documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html)

In [ ]:
#code here

## Scenario

As data scientists, we want to build a model to predict the sale price of a house in Seattle in 2019, based on its square footage. We know that the King County Department of Assessments has comprehensive data available on real property sales in the Seattle area. We need to prepare the data.

### First, get the data!

When working on a project involving data that can fit on our computer, we store it in a `data` directory.

```bash
cd <project_directory>  # example: cd ~/flatiron_ds/pandas-3
mkdir data
cd data
```

Note that `<project_directory>` in angle brackets is a _placeholder_. You should type the path to the actual location on your computer where you're working on this project. Do not literally type `<project_directory>` and _do not type the angle brackets_. You can see an example in the _comment_ to the right of the command above.

Now, we'll need to download the two data files that we need. I will send those zipped files to you in Slack.


Note that there are spaces in the file name. Even though you will *never put spaces in filenames*, you may need to deal with spaces that _other_ people have used in filenames.

There are two ways to handle the spaces in these filenames when referencing them at the command line.

#### 1. You can _escape_ the spaces by putting a backslash (`\`, remember _backslash is next to backspace_) before each one:

`unzip Real\ Property\ Sales.zip`

This is what happens if you tab-complete the filename in the terminal. Tab completion is your friend!

#### 2. You can put the entire filename in quotes:

`unzip "Real Property Sales.zip"`

Try unzipping these files with the `unzip` command. The `unzip` command takes one argument, the name of the file that you want to unzip.

In [20]:
sales_df = pd.read_csv('data/Real Property Sales.zip')

### Seeing pink? Warnings are useful!

Note the warning above: `DtypeWarning: Columns (1, 2) have mixed types.` Because we start with an index of zero, the columns that we're being warned about are actually the _second_ and _third_ columns, `sales_df['Major']` and `sales_df['Minor']`.

In [18]:
sales_df.head().T

,0,1,2,3,4
ExciseTaxNbr,2687551,1235111,2704079,2584094,1056831
Major,138860,664885,423943,403700,951120
Minor,110,40,50,715,900
DocumentDate,08/21/2014,07/09/1991,10/11/2014,01/04/2013,04/20/1989
SalePrice,245000,0,0,0,85000
RecordingNbr,20140828001436,199203161090,20141205000558,20130110000910,198904260448
Volume,,071,,,117
Page,,001,,,053
PlatNbr,,664885,,,951120
PlatType,,C,,,P


In [24]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049216 entries, 0 to 2049215
Data columns (total 24 columns):
ExciseTaxNbr          int64
Major                 object
Minor                 object
DocumentDate          object
SalePrice             int64
RecordingNbr          object
Volume                object
Page                  object
PlatNbr               object
PlatType              object
PlatLot               object
PlatBlock             object
SellerName            object
BuyerName             object
PropertyType          int64
PrincipalUse          int64
SaleInstrument        int64
AFForestLand          object
AFCurrentUseLand      object
AFNonProfitUse        object
AFHistoricProperty    object
SaleReason            int64
PropertyClass         int64
SaleWarning           object
dtypes: int64(7), object(17)
memory usage: 375.2+ MB


### Data overload?

That's a lot of columns. We're only interested in identifying the date, sale price, and square footage of each specific property. What can we do?

In [25]:
sales_df = sales_df[['Major', 'Minor', 'DocumentDate', 'SalePrice']]

In [26]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049216 entries, 0 to 2049215
Data columns (total 4 columns):
Major           object
Minor           object
DocumentDate    object
SalePrice       int64
dtypes: int64(1), object(3)
memory usage: 62.5+ MB


In [27]:
bldg_df = pd.read_csv('data/Residential Building.zip')

/Users/stuartmurphy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Another warning! Which column has index 11?

In [28]:
bldg_df.columns[11]

'ZipCode'

`ZipCode` seems like a potentially useful column. We'll need it to determine which house sales took place in Seattle.

In [29]:
bldg_df.head().T

,0,1,2,3,4
Major,4100,4100,4100,4100,4100
Minor,14,14,165,175,406
BldgNbr,1,2,1,1,1
NbrLivingUnits,1,1,1,1,1
Address,3251 S 148TH ST 98168,3251 S 148TH ST 98168,4001 S 148TH ST 98168,4039 S 148TH ST 98168,15051 MILITARY RD S 98188
BuildingNumber,3251,3251,4001,4039,15051
Fraction,,,,,
DirectionPrefix,S,S,S,S,
StreetName,148TH,148TH,148TH,148TH,MILITARY
StreetType,ST,ST,ST,ST,RD


### So many features!

As data scientists, we should be _very_ cautious about discarding potentially useful data. But, today, we're interested in _only_ the total square footage of each property. What can we do?


In [30]:
bldg_df = bldg_df[['Major', 'Minor', 'SqFtTotLiving', 'ZipCode']]

In [34]:
bldg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514381 entries, 0 to 514380
Data columns (total 4 columns):
Major            514381 non-null int64
Minor            514381 non-null int64
SqFtTotLiving    514381 non-null int64
ZipCode          468541 non-null object
dtypes: int64(3), object(1)
memory usage: 15.7+ MB


In [35]:
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

### Error!

Why are we seeing an error when we try to join the dataframes?

<table>
    <tr>
        <td style="text-align:left"><pre>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013160 entries, 0 to 2013159
Data columns (total 4 columns):
Major           object
Minor           object
DocumentDate    object
SalePrice       int64
dtypes: int64(1), object(3)
memory usage: 61.4+ MB</pre></td>
        <td style="text-align:left"><pre>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511359 entries, 0 to 511358
Data columns (total 4 columns):
Major            511359 non-null int64
Minor            511359 non-null int64
SqFtTotLiving    511359 non-null int64
ZipCode          468345 non-null object
dtypes: int64(3), object(1)
memory usage: 15.6+ MB
</pre></td>
    </tr>
</table>

Review the error message in light of the above:

* `ValueError: You are trying to merge on object and int64 columns.`

In [36]:
sales_df['Major'] = pd.to_numeric(sales_df['Major'])

ValueError: Unable to parse string "      " at position 949114

### Error!

Note the useful error message above:

`ValueError: Unable to parse string "      " at position 936643`

In this case, we want to treat non-numeric values as missing values. Let's see if there's a way to change how the `pd.to_numeric` function handles errors.

In [37]:
# The single question mark means "show me the docstring"
pd.to_numeric?

Here's the part that we're looking for:
```
errors : {'ignore', 'raise', 'coerce'}, default 'raise'
    - If 'raise', then invalid parsing will raise an exception
    - If 'coerce', then invalid parsing will be set as NaN
    - If 'ignore', then invalid parsing will return the input
```

Let's try setting the `errors` parameter to `'coerce'`.

In [38]:
sales_df['Major'] = pd.to_numeric(sales_df['Major'], errors='coerce')

Did it work?

In [39]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049216 entries, 0 to 2049215
Data columns (total 4 columns):
Major           float64
Minor           object
DocumentDate    object
SalePrice       int64
dtypes: float64(1), int64(1), object(2)
memory usage: 62.5+ MB


It worked! Let's do the same thing with the `Minor` parcel number.

In [40]:
sales_df['Minor'] = pd.to_numeric(sales_df['Minor'], errors='coerce')

In [41]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049216 entries, 0 to 2049215
Data columns (total 4 columns):
Major           float64
Minor           float64
DocumentDate    object
SalePrice       int64
dtypes: float64(2), int64(1), object(1)
memory usage: 62.5+ MB


Now, let's try our join again.

In [42]:
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

In [43]:
sales_data

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
0,138860.0,110.0,08/21/2014,245000,1490,98002
1,138860.0,110.0,06/12/1989,109300,1490,98002
2,138860.0,110.0,01/16/2005,14684,1490,98002
3,138860.0,110.0,06/08/2005,0,1490,98002
4,423943.0,50.0,10/11/2014,0,960,98092
5,423943.0,50.0,05/14/1998,0,960,98092
6,423943.0,50.0,07/15/1999,96000,960,98092
7,423943.0,50.0,01/08/2001,127500,960,98092
8,423943.0,50.0,05/21/1998,0,960,98092
9,423943.0,50.0,01/10/2001,0,960,98092


In [44]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1464608 entries, 0 to 1464607
Data columns (total 6 columns):
Major            1464608 non-null float64
Minor            1464608 non-null float64
DocumentDate     1464608 non-null object
SalePrice        1464608 non-null int64
SqFtTotLiving    1464608 non-null int64
ZipCode          1341848 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 78.2+ MB


We can see right away that we're missing zip codes for many of the sales transactions. (1321536 non-null entries for ZipCode is fewer than the 1436772 entries in the dataframe.) 

In [63]:
sales_data.loc[sales_data['ZipCode'].isna()].tail()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
1464543,940672.0,150.0,09/05/2018,564995,2700,NaN
1464554,123310.0,516.0,09/23/2001,599800,3180,NaN
1464575,894676.0,1260.0,05/04/2018,1049995,3470,NaN
1464590,259749.0,790.0,05/05/2013,535990,1800,NaN
1464597,982870.0,2342.0,07/17/2018,1135000,1690,NaN


Because we are interested in finding houses in Seattle zip codes, we will need to drop the rows with missing zip codes.

*Note:* we can use [Boolean Indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing) to select all the rows where there are no missing zip codes

In [46]:
sales_data2 = sales_data.loc[~sales_data['ZipCode'].isna(), :]
sales_data2.head()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
0,138860.0,110.0,08/21/2014,245000,1490,98002
1,138860.0,110.0,06/12/1989,109300,1490,98002
2,138860.0,110.0,01/16/2005,14684,1490,98002
3,138860.0,110.0,06/08/2005,0,1490,98002
4,423943.0,50.0,10/11/2014,0,960,98092


### An alternative method to drop missing is `.dropna()`

In [47]:
sales_data3=sales_data.dropna()
sales_data3.head()

,Major,Minor,DocumentDate,SalePrice,SqFtTotLiving,ZipCode
0,138860.0,110.0,08/21/2014,245000,1490,98002
1,138860.0,110.0,06/12/1989,109300,1490,98002
2,138860.0,110.0,01/16/2005,14684,1490,98002
3,138860.0,110.0,06/08/2005,0,1490,98002
4,423943.0,50.0,10/11/2014,0,960,98092


# Your turn: Data Cleaning with Pandas

### 1. Investigate and drop rows with invalid values in the SalePrice and SqFtTotLiving columns.

Use multiple notebook cells to accomplish this! Press `[esc]` then `B` to create a new cell below the current cell. Press `[return]` to start typing in the new cell.

In [67]:
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)

pd.options.display.max_seq_items = 2000

In [68]:
print(sales_data.SalePrice.value_counts())

0          496691
250000       5085
300000       5052
350000       4867
200000       4399
325000       4250
400000       4249
225000       4244
150000       4124
275000       4033
450000       4007
175000       3721
375000       3641
180000       3601
425000       3593
220000       3592
210000       3569
260000       3565
230000       3559
160000       3512
165000       3508
265000       3426
185000       3419
280000       3374
235000       3370
240000       3358
125000       3349
500000       3329
215000       3310
330000       3296
285000       3285
320000       3274
310000       3259
315000       3205
135000       3173
550000       3158
190000       3068
245000       3063
170000       3060
130000       3055
270000       3021
140000       3005
290000       2993
155000       2968
360000       2967
340000       2951
120000       2924
335000       2868
255000       2851
145000       2840
295000       2834
195000       2812
380000       2792
600000       2750
205000       2750
385000    

In [71]:
sales_data = sales_data.loc[(sales_data['SalePrice']>0) & (sales_data['SqFtTotLiving'] != 0)]

sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 967905 entries, 0 to 1464607
Data columns (total 6 columns):
Major            967905 non-null float64
Minor            967905 non-null float64
DocumentDate     967905 non-null object
SalePrice        967905 non-null int64
SqFtTotLiving    967905 non-null int64
ZipCode          880984 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 51.7+ MB


### 2. Investigate and handle non-numeric ZipCode values

Can you find a way to shorten ZIP+4 codes to the first five digits?

What's the right thing to do with missing values?

In [72]:
# Read the error message and decide how to fix it.
# Note: using errors='coerce' is the *wrong* choice in this case.
def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].head()

165    NaN
166    NaN
167    NaN
169    NaN
170    NaN
Name: ZipCode, dtype: object

### 3. Add a column for PricePerSqFt



In [73]:
sales_data['PricePerSqFt'] = sales_data['SalesPrice'] / sales_data['SqFtTotLiving']

KeyError: 'SalesPrice'

### 4. Subset the data to 2019 sales only.

We can assume that the DocumentDate is approximately the sale date.

### 5. Subset the data to zip codes within the City of Seattle.

You'll need to find a list of Seattle zip codes!

### 6. What is the mean price per square foot for a house sold in Seattle in 2019?

Don't just type the answer. Type code that generates the answer as output!